# Update Program Code Descriptions

In [1]:
import numpy as np
import pandas as pd
from siuba import *

from shared_utils import geography_utils
from dla_utils import _dla_utils

from calitp import to_snakecase

import utils

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)

In [3]:
df = utils.get_clean_data()

In [4]:
df.sample(3)

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name,project_title_new
478,2022-08-09,YS30,Highway Safety Improvement Program (HSIP),5946185,0619000184L,AVENUE 144 AND ROAD 96 ROUNDABOUT. CONVERT INTERSECTION FROM A TWO-WAY STOP INTO ROUNDABOUT.,107.00,Cong Dist 21,16.00,Right of Way,"225,000.00",L5946TCAG,"5,946.00",Tulare County,6.00,Tulare County,Tulare County Association of Governments,Tulare County Association Of Governments,Roundabout in Tulare County
1221,2022-08-31,Y001,National Highway Performance Program (NHPP),2101856,0716000057S,"IN GLENDORA, AT VARIOUS LOCATIONS FROM 0.2 MILE EAST OF S. BARRANCA AVENUE OVERCROSSING TO 0.1 MILE WEST OF AMELIA AVENUE UNDERCROSSING. CONSTRUCT AN",37.00,Cong Dist 27,37.00,Mitigation of Water Pollution due to Highway Runoff,"2,649,914.69",S SCAG,NaN,California,7.00,Los Angeles County,NaN,NaN,Mitigation of Water Pollution due to Highway Runoff in Los Angeles County
611,2022-06-14,YS32,Section 164 Penalties - Use for HSIP Activities,000C532,0519000032S,MONTEREY AND SAN LUIS OBISPO COUNTIES AT VARIOUS LOCATIONS. INSTALL CENTERLINE AND EDGE LINE RUMBLE STRIP,999.00,Cong Dist 0,21.00,Safety,"1,831,000.00",S SLOCOG,NaN,California,NaN,Statewide,NaN,NaN,Safety Improvements Statewide


In [71]:
df>>group_by(_.project_number)>>summarize(n = _.district.nunique())>>arrange(-_.n)

,project_number,n
57,0101856,2
445,P035012,2
446,P036118,2
459,P057069,2
465,P060162,2
...,...,...
451,P050156,0
532,X001665,0
533,X019032,0
540,X059083,0


## Condense Projects into One

In [27]:
df_agg = df.astype(str).groupby('project_number').agg(' | ' .join).reset_index()

In [30]:
df_agg.sample()

,project_number,fmis_transaction_date,program_code,program_code_description,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name,project_title_new
181,5133045,2022-07-14 | 2022-07-14,YS30 | YS30,Highway Safety Improvement Program (HSIP) | Highway Safety Improvement Program (HSIP),1218000021L | 1218000021L,FOURTEEN (14) SELECT INTERSECTIONS CITYWIDE INSTALL FOURTEEN (14) PEDESTRIAN WARNING FLASHING BEACONS CITYWIDE | FOURTEEN (14) SELECT INTERSECTIONS CITYWIDE INSTALL FOURTEEN (14) PEDESTRIAN WARNING FLASHING BEACONS CITYWIDE,59.0 | 59.0,Cong Dist 39 | Cong Dist 39,17.0 | 21.0,Construction Engineering | Safety,1000.0 | 285200.0,L5133SCAG | L5133SCAG,5133.0 | 5133.0,Fullerton | Fullerton,12.0 | 12.0,Orange County | Orange County,Orange County Transportation Authority | Orange County Transportation Authority,Southern California Association Of Governments | Southern California Association Of Governments,Install Pedestrian Safety Improvements in Fullerton | Install Pedestrian Safety Improvements in Fullerton


In [40]:
(list(df.columns))

['fmis_transaction_date',
 'program_code',
 'program_code_description',
 'project_number',
 'recipient_project_number',
 'project_title',
 'county_code',
 'congressional_district',
 'improvement_type',
 'improvement_type_description',
 'obligations_amount',
 'summary_recipient_defined_text_field_1_value',
 'implementing_agency_locode',
 'implementing_agency',
 'district',
 'county_name',
 'rtpa_name',
 'mpo_name',
 'project_title_new']

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1241 entries, 0 to 1211
Data columns (total 19 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   fmis_transaction_date                         1241 non-null   datetime64[ns]
 1   program_code                                  1241 non-null   object        
 2   program_code_description                      1240 non-null   object        
 3   project_number                                1241 non-null   object        
 4   recipient_project_number                      1240 non-null   object        
 5   project_title                                 1241 non-null   object        
 6   county_code                                   1241 non-null   float64       
 7   congressional_district                        1241 non-null   object        
 8   improvement_type                              1241 non-null   float6

In [49]:
df2 = df.copy()

In [78]:
df2[['county_code', 'improvement_type', 'implementing_agency_locode', 'district', 'program_code_description']] = df2[['county_code', 'improvement_type', 'implementing_agency_locode', 'district', 'program_code_description']].astype(str)

In [105]:
# # doesnt work
# df_agg2 = (df2.groupby(['project_number', 'fmis_transaction_date', 'project_title_new', 'implementing_agency'], as_index=False)
#            .agg({'obligations_amount':'sum', 
#                  'recipient_project_number':lambda x: ' | '.join(x),
#                  'program_code':lambda x: ' | '.join(x),
#                  'program_code_description':lambda x: ' | '.join(x),
#                  'county_code':lambda x: ' | '.join(x),
#                  'congressional_district':lambda x: ' | '.join(x),
#                  'improvement_type':lambda x: ' | '.join(x),
#                  'improvement_type_description':lambda x: ' | '.join(x)
#                 }))


In [130]:
df_agg = (df2
           .assign(count=1)
           .groupby(['fmis_transaction_date','project_number', 'implementing_agency', 'summary_recipient_defined_text_field_1_value'])
           .agg({'recipient_project_number':'first',
                 'improvement_type':lambda x:' | '.join(x.unique()),
                 'improvement_type_description':lambda x:' | '.join(x.unique()),
                 'program_code':lambda x:' | '.join(x.unique()),
                 'program_code_description':lambda x:' | '.join(x.unique()),
                 'obligations_amount':'sum',
                 'congressional_district':lambda x:' | '.join(x.unique()),
                 'district':lambda x:' | '.join(x.unique()),
                 'project_title':'first',
                 'county_code':lambda x:' | '.join(x.unique()),
                 'county_name':lambda x:' | '.join(x.unique()),
                 'implementing_agency_locode':lambda x:' | '.join(x.unique()),
                 'rtpa_name':'first',
                 'mpo_name':'first',
                 'project_title_new':'first'
                }).reset_index())

In [132]:
df_agg.sample(5)

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,recipient_project_number,improvement_type,improvement_type_description,program_code,program_code_description,obligations_amount,congressional_district,district,project_title,county_code,county_name,implementing_agency_locode,rtpa_name,mpo_name,project_title_new
17,2022-02-15,5928128,Contra Costa County,L5928MTC,0416000343L,11.0 | 17.0,Bridge Replacement - No Added Capacity | Construction Engineering,Y001,National Highway Performance Program (NHPP),"8,482,104.00",Cong Dist 11,4.0,MARSH DRIVE BRIDGE NO. 28C-0442 OVER THE WALNUT CREEK CHANNEL REPLACE EXISTING TWO-LANE BRIDGE WITH A NEW TWO-LANE BRIDGE,13.0,Contra Costa County,5928.0,Metropolitan Transportation Commission,Metropolitan Transportation Commission,Bridge Replacement in Contra Costa County
12,2022-02-14,5941104,Madera County,L5941MCTC,0615000093L,11.0 | 15.0 | 17.0,Bridge Replacement - No Added Capacity | Preliminary Engineering | Construction Engineering,Y233,Surface Transportation Block Grant (STBG) Program,"2,110,500.00",Cong Dist 16,6.0,BRIDGE 41C0099 COUNTY ROAD 5 1/2 OVER CHOWCHILLA CREEK REPLACE 2 LANE BRIDGE WITH 2 LANE BRIDGE (TC),39.0,Madera County,5941.0,Madera County Transportation Commission - RTPA,Madera County Transportation Commission,Replace Bridge Replacement in Madera County
442,2022-09-08,5059258,Modesto,L5059STANCOG,1022000269L,3.0,4R - Added Capacity,Y400,Congestion Mitigation & Air Quality Improvement (CMAQ),"1,200,000.00",Cong Dist 10,10.0,"INTERSECTION OF PELANDALE AVENUE AND SR 99 IN MODESTO, CA TO INSTALL A SECOND LEFT TURN LANE FROM EASTBOUND PELANDALE AVENUE TO NORTHBOUND SISK ROAD",99.0,Stanislaus County,5059.0,Stanislaus Council of Governments,Stanislaus Council Of Governments,Install Turn Lane in Modesto
482,2022-09-13,5060357,Fresno,L5060FCOG,0619000125L,17.0 | 21.0,Construction Engineering | Safety,YS30,Highway Safety Improvement Program (HSIP),"2,014,500.00",Cong Dist 16,6.0,"FRESNO STREET AND THOMAS AVENUE INTERSECTIONS, FRESNO STREET AND SAN JOSE AVENUE INTERSECTION, FRESNO AND R STREETS INTERSECTION, FRESNO STREET AND C",19.0,Fresno County,5060.0,Council of Fresno County Governments,Council Of Fresno County Goverments,Safety Improvements in Fresno
123,2022-05-05,6204134,Caltrans,S6204MTC,0419000044L,15.0,Preliminary Engineering,Y230,Surface Transportation Block Grant (STBG) Program,"1,740,000.00",Cong Dist 13,4.0,"ALONG THE I-880 CORRIDOR SF BAY AREA: REGIONWIDE: IMPLEMENT A COLLECTIVE APPROACH TO FREEWAY OPERATIONS AND MANAGEMENT, INCLUDING COMMUNICATIONS NETW",1.0,Multi-County,6204.0,CT-ADMIN,CT-ADMIN,Preliminary Engineering Projects in Caltrans


In [133]:
len(df_agg)

552